In [12]:
import numpy as np
from MLRNet import *
from mlrnet_architectures import *
from copy import deepcopy
import torch
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import roc_auc_score as auc

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
seed = 0

# Boston Dataset

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.datasets import load_boston as lb
def prepare_dataset(X, y, train_size = 0.8, normalize_X = True, seed= False):
    kwargs = {}
    if seed or type(seed)== type(0):
        kwargs["random_state"] = seed
        np.random.seed(seed)
    if normalize_X: X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = tts(X, y, train_size = train_size, **kwargs)
    return X_train, X_test, y_train, y_test
def prepare_boston(train_size = 0.8, seed= 0, normalize_X = True, standardize_y = True):
    X,y = lb(return_X_y=True)
    if standardize_y: y = (y - y.mean())/y.std()
    X_train, X_test, y_train, y_test = prepare_dataset(X, y, train_size = train_size , seed=seed, normalize_X = normalize_X)
    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = prepare_boston(seed = seed, standardize_y = False)

# Check Random Forests

In [5]:
from sklearn.ensemble import RandomForestRegressor as RF
reg = RF(n_estimators = 1000, random_state = seed)
reg.fit(X_train,y_train)
print("RF","R2-score :",reg.score(X_test, y_test))

RF R2-score : 0.7685762389779094


# Define basic nn architectures

In [6]:
import architectures
mlrnetfastv3 = {"lr_scheduler" : "OneCycleLR",
                "lr_scheduler_params" : {"max_lr":1e-2, "total_steps" : 200},
                "max_iter":200,
                "learning_rate":1e-3, #useless with OneCycleLR
                "hidden_nn" : architectures.DenseLayers,
                "hidden_params" :  {"width":256,"depth":2, "device":device}   #2layers, 256 neurons per layer, 200 iterations        
                }
mlrnetslowv2 = {"lr_scheduler" : "OneCycleLR",
                "lr_scheduler_params" : {"max_lr":1e-3, "total_steps" : 500},
                "max_iter":500,
                "learning_rate":1e-3, #useless with OneCycleLR
                "hidden_nn" : architectures.DenseLayers,
                "hidden_params" :  {"width":1024,"depth":2, "device":device} #2layers, 1024 neurons per layer, 500 iterations     
                }

# MLRNet

In [7]:
default_parameters = deepcopy(eval("mlrnetfastv3"))
default_parameters["hidden_params"]["n_features"] = X_train.shape[-1]
reg = MLRNetRegressor(random_state = seed, **default_parameters)
reg.fit(X_train,y_train)
print("MLRNet-fast","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights()  #else gpu memory leaks
del reg

MLRNet-fast R2-score : 0.7933839775958988


In [8]:
default_parameters = deepcopy(eval("mlrnetslowv2"))
default_parameters["hidden_params"]["n_features"] = X_train.shape[-1]
reg = MLRNetRegressor(random_state = seed, **default_parameters)
reg.fit(X_train,y_train)
print("MLRNet-wide","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights()  #else gpu memory leaks
del reg

MLRNet-wide R2-score : 0.8184638398794895


# Regular Net, same architecture + BatchNorm & DropOut

In [9]:
NN_params = deepcopy(eval("mlrnetfastv3"))
NN_params["closeform_parameter_init"] = False #No closeform
NN_params["hidden_params"].update({"dropout":0.1, "batch_norm":True, "output":1})
NN_params["hidden_params"]["n_features"] = X_train.shape[-1]

reg = MLRNetRegressor(random_state = seed, **NN_params)
reg.fit(X_train,y_train)
print("Regular Net-fast","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights()  #else gpu memory leaks
del reg

Regular Net-fast R2-score : 0.7983281142962326


In [10]:
NN_params = deepcopy(eval("mlrnetslowv2"))
NN_params["closeform_parameter_init"] = False #No closeform
NN_params["hidden_params"].update({"dropout":0.1, "batch_norm":True, "output":1})
NN_params["hidden_params"]["n_features"] = X_train.shape[-1]

reg = MLRNetRegressor(random_state = seed, **NN_params)
reg.fit(X_train,y_train)
print("Regular Net-wide","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights()  #else gpu memory leaks
del reg

Regular Net-wide R2-score : 0.7625664811538111
